In [92]:
import pickle
import os
import time

from docutils.nodes import container

df = pickle.load(open(os.path.join('./datasets/flow/Medboit_all_seq_30_2.pickle'), 'rb'))
df.shape

FileNotFoundError: [Errno 2] No such file or directory: './datasets/flow/Medboit_all_seq_30_2.pickle'

In [2]:
from data_load.get_iot import get_iot
from flowcontainer.extractor import extract
from tqdm import *
all_address, all_label = get_iot()
ans = 0
for i in trange(len(all_address)):
    result = extract(all_address[i], filter='tcp or udp')
    ans += len(result)
ans

100%|██████████| 21/21 [09:00<00:00, 25.75s/it]


256849

In [4]:
from data_load.get_ustctfc import get_ustctfc
all_address, all_label = get_ustctfc()
ans = 0
for i in trange(len(all_address)):
    result = extract(all_address[i], filter='tcp or udp')
    ans += len(result)
ans

./datasets/pcap/USTC-TFC2016/Benign/MySQL.pcap 0
./datasets/pcap/USTC-TFC2016/Benign/FTP.pcap 1
./datasets/pcap/USTC-TFC2016/Benign/BitTorrent.pcap 2
./datasets/pcap/USTC-TFC2016/Benign/Skype.pcap 3
./datasets/pcap/USTC-TFC2016/Benign/Gmail.pcap 4
./datasets/pcap/USTC-TFC2016/Benign/Facetime.pcap 5
./datasets/pcap/USTC-TFC2016/Benign/Outlook.pcap 6
./datasets/pcap/USTC-TFC2016/Benign/WorldOfWarcraft.pcap 7
./datasets/pcap/USTC-TFC2016/Malware/Geodo.pcap 8
./datasets/pcap/USTC-TFC2016/Malware/Cridex.pcap 9
./datasets/pcap/USTC-TFC2016/Malware/Miuref.pcap 10
./datasets/pcap/USTC-TFC2016/Malware/Virut.pcap 11
./datasets/pcap/USTC-TFC2016/Malware/Neris.pcap 12
./datasets/pcap/USTC-TFC2016/Malware/Zeus.pcap 13
./datasets/pcap/USTC-TFC2016/Malware/Shifu.pcap 14
./datasets/pcap/USTC-TFC2016/Malware/Nsis-ay.pcap 15
./datasets/pcap/USTC-TFC2016/Malware/Htbot.pcap 16
./datasets/pcap/USTC-TFC2016/Malware/Tinba.pcap 17


 61%|██████    | 11/18 [00:33<00:24,  3.47s/it]/home/rtx4090/anaconda3/lib/python3.12/site-packages/flowcontainer/reader.py:218: UserWarning: Error reading file: ' ** (tshark:3240416) 17:22:18.742066 [Epan WARNING] -- Dissector bug, protocol TLS, in packet 1746: ./epan/dissectors/packet-tls-utils.c:6412: failed assertion "offset <= offset_end"
 ** (tshark:3240416) 17:22:19.506816 [Epan WARNING] -- Dissector bug, protocol TLS, in packet 154811: ./epan/dissectors/packet-tls-utils.c:6412: failed assertion "offset <= offset_end"
 ** (tshark:3240416) 17:22:20.288737 [Epan WARNING] -- Dissector bug, protocol TLS, in packet 1746: ./epan/dissectors/packet-tls-utils.c:6412: failed assertion "offset <= offset_end"
 ** (tshark:3240416) 17:22:22.287327 [Epan WARNING] -- Dissector bug, protocol TLS, in packet 154811: ./epan/dissectors/packet-tls-utils.c:6412: failed assertion "offset <= offset_end"
'
  warnings.warn("Error reading file: '{}'".format(
 67%|██████▋   | 12/18 [00:40<00:27,  4.59s/it]/

297359

In [9]:
from data_load.get_medboit import get_medboit
all_address, all_label = get_medboit()
ans = 0
for i in trange(len(all_address)):
    result = extract(all_address[i], filter='tcp or udp')
    ans += len(result)
ans

./datasets/pcap/Medboit/mirai_mal_CC_all.pcap 1
./datasets/pcap/Medboit/bashlite_mal_CC_all.pcap 2
./datasets/pcap/Medboit/torii_leg.pcap 0
./datasets/pcap/Medboit/bashlite_leg.pcap 0
./datasets/pcap/Medboit/bashlite_mal_spread_all.pcap 3
./datasets/pcap/Medboit/torii_mal_all.pcap 4
./datasets/pcap/Medboit/mirai_leg.pcap 0
./datasets/pcap/Medboit/mirai_mal_spread_all.pcap 5


100%|██████████| 8/8 [13:01<00:00, 97.70s/it] 


296805

In [6]:
# 

In [7]:
import subprocess
import os 
def split_pcap_by_command(dataset, src_address, label):
    dst_address = f'./datasets/split_cap/{dataset}/{src_address.split('/')[-1].split('.')[0]}'
    cmd = f"mono SplitCap.exe -r {src_address} -s session -o '{dst_address}'" 
    # print(cmd)
    # 执行命令
    subprocess.run(cmd, shell=True, check=True)
    pcap_name = src_address.split('/')[-1].split('.')[0]
    x = os.listdir(f'./datasets/split_cap/{dataset}/{pcap_name}')
    y = []
    for i in range(len(x)):
        x[i] = f'./datasets/split_cap/{dataset}/{pcap_name}/' + x[i]
        y.append(label)
    return x, y

In [8]:
import multiprocessing
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm
from flowcontainer.extractor import extract

# 你的数据
x, y = [], []

# 你的处理函数
def process_address(address, label):
    feature_result = extract(address, filter='tcp or udp', extension=['tls.record.content_type', 'tls.record.opaque_type', 'tls.handshake.type'])
    flow_lengths = []
    for key, value in feature_result.items():
        flow_lengths.extend(value.ip_lengths)
    # if len(flow_lengths) < 30:
    #     return None  # 过滤掉不符合条件的数据
    return flow_lengths[:30], label

# 使用进程池进行并行计算
num_workers = min(multiprocessing.cpu_count(), len(all_address))  # 限制最大线程数
with ProcessPoolExecutor(max_workers=num_workers) as executor:
    # 提交任务
    future_to_index = {executor.submit(process_address, all_address[i], all_label[i]): i for i in range(len(all_address))}
    # 使用 tqdm 进度条，遍历完成的任务
    results = []
    for future in tqdm(as_completed(future_to_index), total=len(all_address)):
        result = future.result()
        if result is not None:
            results.append(result)

100%|██████████| 8/8 [07:51<00:00, 58.92s/it] 


In [48]:
a, b = zip(*results)

In [12]:
import os
def get_medboit():
    all_address, all_label = [], []
    benign_address = [item for item in os.listdir('./datasets/pcap/Medboit') if 'pcap' in item]
    cnt = 1
    for i in range(len(benign_address)):
        benign_address[i] = './datasets/pcap/Medboit/' + benign_address[i]
        all_address.append(benign_address[i])
        if 'leg' in benign_address[i]:
            all_label.append(0)
        else:
            all_label.append(cnt)
            cnt += 1

    for i in range(len(all_address)):
        print(all_address[i], all_label[i])
    return all_address, all_label
all_address, all_label = get_medboit()

./datasets/pcap/Medboit/mirai_mal_CC_all.pcap 1
./datasets/pcap/Medboit/bashlite_mal_CC_all.pcap 2
./datasets/pcap/Medboit/torii_leg.pcap 0
./datasets/pcap/Medboit/bashlite_leg.pcap 0
./datasets/pcap/Medboit/bashlite_mal_spread_all.pcap 3
./datasets/pcap/Medboit/torii_mal_all.pcap 4
./datasets/pcap/Medboit/mirai_leg.pcap 0
./datasets/pcap/Medboit/mirai_mal_spread_all.pcap 5
